# 04 Machine Learning Pipelines

View: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [36]:
eucalyptus = pd.read_csv("../datasets/eucalyptus.csv")

In [37]:
eucalyptus

,Abbrev,Rep,Locality,Map_Ref,Latitude,Altitude,Rainfall,Frosts,Year,Sp,PMCno,DBH,Ht,Surv,Vig,Ins_res,Stem_Fm,Crown_Fm,Brnch_Fm,Utility
0,Cra,1,Central_Hawkes_Bay,N135_382/137,39__38,100.0,850.0,-2.0,1980.0,co,1520.0,18.45,9.96,40.0,4.0,3.0,3.5,4.0,3.5,good
1,Cra,1,Central_Hawkes_Bay,N135_382/137,39__38,100.0,850.0,-2.0,1980.0,fr,1487.0,13.15,9.65,90.0,4.5,4.0,3.5,3.5,3.0,best
2,Cra,1,Central_Hawkes_Bay,N135_382/137,39__38,100.0,850.0,-2.0,1980.0,ma,1362.0,10.32,6.50,50.0,2.3,2.5,3.0,3.5,3.0,low
3,Cra,1,Central_Hawkes_Bay,N135_382/137,39__38,100.0,850.0,-2.0,1980.0,nd,1596.0,14.80,9.48,70.0,3.7,3.0,3.3,4.0,3.5,good
4,Cra,1,Central_Hawkes_Bay,N135_382/137,39__38,100.0,850.0,-2.0,1980.0,ni,2088.0,14.50,10.78,90.0,4.0,2.7,3.3,3.0,3.0,good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,WSh,1,Southern_Hawkes_Bay,N151_922/226,40__36,100.0,1250.0,-2.0,1983.0,fa,2548.0,41.63,12.64,28.0,4.2,3.2,2.3,1.9,1.7,average
732,WSh,1,Southern_Hawkes_Bay,N151_922/226,40__36,100.0,1250.0,-2.0,1983.0,fr,2552.0,33.35,10.61,33.0,4.5,4.0,2.8,3.0,1.5,good
733,WSh,1,Southern_Hawkes_Bay,N151_922/226,40__36,100.0,1250.0,-2.0,1983.0,ni,2568.0,28.21,9.47,94.0,4.6,3.0,2.0,1.8,1.2,good
734,WSh,1,Southern_Hawkes_Bay,N151_922/226,40__36,100.0,1250.0,-2.0,1983.0,ob,1522.0,27.36,11.49,67.0,4.7,3.3,3.4,3.4,3.0,good


In [38]:
pipeline = Pipeline(
    [
        ("polynomical", PolynomialFeatures()),
        ('pca', PCA()),
        ('dt',DecisionTreeClassifier())
    ]
)

pipeline.fit(eucalyptus.drop(columns=['Utility']),eucalyptus['Utility'])

ValueError: could not convert string to float: 'Cra'

Ejemplo con otra base de datos.

In [39]:
df = pd.DataFrame(data={
    "x1":['a','b','b',np.nan],
    'x2':[1,np.nan,2,3]
})

In [40]:
df

,x1,x2
0,a,1.0
1,b,NaN
2,b,2.0
3,NaN,3.0


En la base de datos anterior no podemos aplicar un Pipeline puesto que hay valores nulos, si hacemos un transform nos va a fallar, por esta razón debemos hacer imputación:

In [41]:
from sklearn.impute import SimpleImputer, KNNImputer

In [42]:
SimpleImputer(strategy='most_frequent').fit_transform(df) 

array([['a', 1.0],
       ['b', 1.0],
       ['b', 2.0],
       ['b', 3.0]], dtype=object)

In [43]:
pl=Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder())
])

Pero no siempre funciona...

In [44]:
SimpleImputer(strategy='constant',fill_value=0).fit_transform(df)

array([['a', 1.0],
       ['b', 0],
       ['b', 2.0],
       [0, 3.0]], dtype=object)

En el caso anterior hace una imputación en la primera columna con un número cuando realmente debe ser un string. Si aplicamos OneHotEncoder generaría una columna adicional. Esto esta mal.

## Column Transformers
https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

Ahora tenemos lo siguiente:

transformers: list of tuples  
List of (name, transformer, columns) tuples specifying the transformer objects to be applied to subsets of the data.

In [45]:
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [46]:
imputer=make_column_transformer(
    (SimpleImputer(strategy='median'),['x2']),
    (SimpleImputer(strategy='most_frequent'),['x1'])
)

imputer.fit_transform(df)

array([[1.0, 'a'],
       [2.0, 'b'],
       [2.0, 'b'],
       [3.0, 'b']], dtype=object)

Ya obtenemos un buen df, hizo la imputación correctamente. Sin embargo, es importante notar que el resultado es un numpy array, no un dataframe. Es muy importante notar esto porque **perdemos la posibilidad de trabajar con nombres de columnas.**

Otro cambio importante a notar es el hecho de que cambio el orden de las columnas, esto porque yo primero llame el imputer con `x2`, esto puede generar problemas luego. 

```ad-note
Cuando trabajamos con datos, es importante empezar a desprenderse de los dataframes, realmente lo que usamos más son numpy arrays o tensores. 
```

si ahora quiero hacer más pasos:

In [47]:
combined_steps=make_column_transformer(
    (SimpleImputer(strategy='median'),['x2']),
    (SimpleImputer(strategy='most_frequent'),['x1']),
    (OneHotEncoder(),['x1'])
)

combined_steps.fit_transform(df)

array([[1.0, 'a', 1.0, 0.0, 0.0],
       [2.0, 'b', 0.0, 1.0, 0.0],
       [2.0, 'b', 0.0, 1.0, 0.0],
       [3.0, 'b', 0.0, 0.0, 1.0]], dtype=object)

Encontramos que el OneHotEncoder resultó en 3 columnas (como si tomara el NaN como categoria), esto es un error en la lógica. Este `Column Transformer` toma el df original y hace las transformaciones sobre esto. Es todo en paralelo, esto explica por qué hay un "x1" original y otro con OneHotEncoder.

El orden de las operaciones es:

1. Numeric  
2. Categorical  
3. One Hot Encoder  

Sin embargo, entre sí no se hablan.

Por esta razón, se deben hacer varios steps. Así nos aseguramos que todo se va ejecutando en el orden que debe ser.

## Pipeline

In [48]:
from sklearn.pipeline import Pipeline

In [49]:
pl=Pipeline([
    ('imputer',imputer),
    ('binarizer',OneHotEncoder())
])

print(pl.fit_transform(df))

  (0, 0)	1.0
  (0, 3)	1.0
  (1, 1)	1.0
  (1, 4)	1.0
  (2, 1)	1.0
  (2, 4)	1.0
  (3, 2)	1.0
  (3, 4)	1.0


Lo anterior nos dice solo en que posiciones tengo valores diferentes de cero. Para solucinar esto realmente el OneHotEncoder debe aplicarse con un column transformer. Asi que obtenemos:

In [50]:
pl=Pipeline([
    ('imputer',imputer),
    ('binarizer',make_column_transformer((OneHotEncoder(),[1])) )
])

print(pl.fit_transform(df))

[[1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


Sin embargo, encontramos otro problema. Ahora ya no tenemos la columna numérica...

Esto es porque en este siguiente paso metimos todo y el resultado es solamente lo que obtiene al final.

Aqui es cuando podemos utilizar el parámetro `remainder='passthrough'`

In [51]:
pl=Pipeline([
    ('imputer',imputer),
    ('binarizer',make_column_transformer((OneHotEncoder(),[1]),remainder='passthrough') )
])

print(pl.fit_transform(df))

[[1.0 0.0 1.0]
 [0.0 1.0 2.0]
 [0.0 1.0 2.0]
 [0.0 1.0 3.0]]
